In [0]:
%run "../includes/configuration"

In [0]:
team_aggregated_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
from pyspark.sql.functions import sum, count, when ,col

In [0]:
team_aggregated_df = team_aggregated_df.groupBy("race_year", "team") \
.agg(sum("points").alias("Points"), 
     count(when(col("position") == 1, True)).alias("wins"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

window = Window.partitionBy("race_year") \
    .orderBy(desc("points"), desc("wins"))
    
final_df = team_aggregated_df.withColumn("rank", rank().over(window))

In [0]:
display(final_df.filter("race_year = 2020"))

In [0]:
final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/costructors_standings")